## Testing Async Feature vs Sync
This notebook demonstrates how to deploy and test a Nuclio function with async feature.
It configures the Nuclio function to handle incoming events, and allocates workers with multiple connections to handle the requests in async mode. 

In [1]:
seimport mlrun
import time
import inspect

In [2]:
project = mlrun.get_or_create_project("async-nuclio-test",user_project=True,allow_cross_project=True)

> 2025-11-09 10:51:57,715 [info] Project loaded successfully: {"project_name":"async-nuclio-test3-guyl"}


In [3]:
%%writefile async_nuclio.py
import asyncio
import json
from datetime import datetime

async def handler(context, event):
    body = event.body

    # handle all three cases: dict, bytes, str
    if isinstance(body, dict):
        data = body
    else:
        if isinstance(body, bytes):
            body = body.decode("utf-8")
        try:
            data = json.loads(body)
        except Exception:
            data = {"raw": body}

    worker = context.worker_id
    context.logger.info(f"Event: {data}, {event.path} with headers: {event.headers}")

    await asyncio.sleep(1)

    resp = {
        "test": data.get("test"),
        "send_at": data.get("send_at"),
        "worker_id": worker,
        "end_time": datetime.now().isoformat(),
    }

    return context.Response(
        body=json.dumps(resp),
        headers={"Content-Type": "application/json"},
        status_code=200,
    )


Overwriting async_nuclio3.py


## Test Nuclio Async Mode
 Testing with the following parameters: `events 20`, `maxConnectionsNumber 20`, `1 worker`, `connectionAvailabilityTimeout 60s`. Testing a number of events distributed to 1 worker, and see the responses time for each event, over all time took to finish handling events and the success and failed hadlings of events. The events should be recieved in the same time and handled at the same time.

In [4]:
async_func = project.set_function(func="async_nuclio.py",name="async-nuclio",image="mlrun/mlrun",kind="nuclio")

In [5]:
async_func.with_http(workers=1, worker_timeout=30)
async_func.spec.min_replicas = 1
async_func.spec.max_replicas = 1
async_func.spec.config["spec.triggers.http"]["mode"] = "async"
async_func.spec.config["spec.triggers.http"]["async"]={"maxConnectionsNumber": 20,"minConnectionsNumber":20, "connectionAvailabilityTimeout":"60s"}
async_func.set_config("spec.eventTimeout", "50s")

> 2025-11-09 10:51:57,739 [warning] Adding HTTP trigger despite the default HTTP trigger creation being disabled


In [6]:
addr = async_func.deploy()

> 2025-11-09 10:51:57,751 [info] Starting remote function deploy
2025-11-09 10:51:58  (info) Deploying function
2025-11-09 10:51:58  (info) Building
2025-11-09 10:51:58  (info) Staging files and preparing base images
2025-11-09 10:51:58  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-11-09 10:51:58  (info) Building processor image
2025-11-09 10:53:13  (info) Build complete
2025-11-09 10:53:23  (info) Function deploy complete
> 2025-11-09 10:53:28,828 [info] Successfully deployed function: {"external_invocation_urls":["async-nuclio-test3-guyl-async-nuclio3.default-tenant.app.cust-cs-il.iguazio-cd0.com/"],"internal_invocation_urls":["nuclio-async-nuclio-test3-guyl-async-nuclio3.default-tenant.svc.cluster.local:8080"]}


'http://async-nuclio-test3-guyl-async-nuclio3.default-tenant.app.cust-cs-il.iguazio-cd0.com/'

In [7]:
import asyncio
import aiohttp
from datetime import datetime
import pandas as pd

url = addr

async def fetch(session, url, i):
    payload = {"test": i, "send_at": datetime.now().isoformat()}
    try:
        async with session.post(url, json=payload) as resp:
            text = await resp.text()
            return {
                "ok": resp.status == 200,
                "status": resp.status,
                "text": text,
                "i": i,
            }
    except Exception as e:
        return {
            "ok": False,
            "status": None,
            "text": str(e),
            "i": i,
        }

async def fetch(session, url, i):
    payload = {"test": i, "send_at": datetime.now().isoformat()}
    try:
        async with session.post(url, json=payload) as resp:
            text = await resp.text()
            return {
                "ok": resp.status == 200,
                "status": resp.status,
                "text": text,
                "i": i,
            }
    except Exception as e:
        return {
            "ok": False,
            "status": None,
            "text": str(e),
            "i": i,
        }

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url, i) for i in range(20)]
        responses = await asyncio.gather(*tasks, return_exceptions=True)

    # do not count here, just return them all
    return responses

responses = await main()


In [9]:
df = pd.DataFrame(responses)

In [11]:
import pandas as pd
import json

# extract both fields from the JSON string
df["send_at"] = df["text"].apply(lambda t: json.loads(t)["send_at"])
df["end_time"] = df["text"].apply(lambda t: json.loads(t)["end_time"])

# Normalize to real datetime objects
df["send_at"] = pd.to_datetime(df["send_at"])
df["end_time"] = pd.to_datetime(df["end_time"])

# Align timestamps to 1 second precision
df["send_at_0_1s"] = df["send_at"].dt.round("1000ms")
df["end_time_0_1s"] = df["end_time"].dt.round("1000ms")

# Verify all values are effectively the same second
assert df["send_at_0_1s"].nunique() == 1, f"send_at mismatch: {df['send_at_0_1s'].unique()}"
assert df["end_time_0_1s"].nunique() == 1, f"end_time mismatch: {df['end_time_0_1s'].unique()}"

# Success check on the HTTP-level result we stored as 'ok'
assert df["ok"].all(), f"Some requests failed: {df[~df['ok']]}"
failed_mask = ~df["ok"]
failed_count = failed_mask.sum()
assert failed_count == 0, f"{failed_count} requests are not True, indexes: {df.index[failed_mask].tolist()}"

print("✅ All checks passed: send_at and end_time align to 0.1s, and all responses are OK.")

✅ All checks passed: send_at and end_time align to 0.1s, and all responses are OK.
